# Full audio

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
n_mfcc = 13

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')


coswara_feature = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coswara_features.npy')
coughvid_feature = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features.npy')

### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,...,stridor_3,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,cough_path,label,error
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,healthy,0
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,healthy,0
2,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0012c608-33d0-4ef7-bde3-75a0b1a0024e.webm,NaN,0
3,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,NaN,NaN,21.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm,healthy,0
4,001c85a8-cc4d-4921-9297-848be52d4715,2020-04-17T15:24:35.822355+00:00,0.0735,40.6,-3.6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001c85a8-cc4d-4921-9297-848be52d4715.webm,NaN,0


In [ ]:
logger.info(meta_df.shape)

2022-04-30 12:14:11,871 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - (20072, 43)


In [ ]:
mfcc_feature = np.load(coughvid_feature)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [ ]:
mfcc_feature[0].shape, len(mfcc_feature), meta_df.shape

((234, 246), 20072, (20072, 43))

In [ ]:
DROP_COLS = [
             'uuid',
             'error',
             'datetime',
             'cough_path',
             'cough_detected',
             ]

meta_df.isna().sum()

uuid                         0
datetime                     0
cough_detected               0
latitude                  8326
longitude                 8326
age                       9604
gender                    8758
respiratory_condition     8758
fever_muscle_pain         8758
status                    8758
quality_1                19252
cough_type_1             19252
dyspnea_1                19252
wheezing_1               19252
stridor_1                19252
choking_1                19252
congestion_1             19252
nothing_1                19252
diagnosis_1              19252
severity_1               19252
quality_2                19252
cough_type_2             19253
dyspnea_2                19252
wheezing_2               19252
stridor_2                19252
choking_2                19252
congestion_2             19252
nothing_2                19252
diagnosis_2              19252
severity_2               19252
quality_3                19252
dyspnea_3                19252
wheezing

In [ ]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
status                    object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3 

In [ ]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
status                   0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
c

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [ ]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-04-30 12:14:18,355 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - {0.0: 8562, 1.0: 1742}


In [ ]:
meta_df.label.value_counts().sum()

10304

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-30 12:14:19,107 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - 1742, 8562


In [ ]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [ ]:
print(len(feature_1))

1742


In [ ]:
del mfcc_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:6850], feature_1[:1394]), axis=0)
  val = np.concatenate((feature_0[6850:7706], feature_1[1394:1568]), axis=0)
  test = np.concatenate((feature_0[7706:], feature_1[1568:]), axis=0)

  y_train = pd.concat([label0[:6850], label1[:1394]], ignore_index=True)
  y_val = pd.concat([label0[6850:7706], label1[1394:1568]], ignore_index=True)
  y_test = pd.concat([label0[7706:], label1[1568:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(8244, 234, 246)


In [ ]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
X_train_1d.shape

(8244, 37, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 84s 63ms/step - loss: 0.5060 - auc: 0.8355 - val_loss: 0.4589 - val_auc: 0.8376
Epoch 2/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4652 - auc: 0.8384 - val_loss: 0.4722 - val_auc: 0.8299
Epoch 3/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4600 - auc: 0.8387 - val_loss: 0.4583 - val_auc: 0.8474
Epoch 4/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4586 - auc: 0.8421 - val_loss: 0.4661 - val_auc: 0.8359
Epoch 5/40
1031/1031 [==============================] - 61s 59ms/step - loss: 0.4563 - auc: 0.8431 - val_loss: 0.4627 - val_auc: 0.8292
Epoch 6/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4542 - auc: 0.8447 - val_loss: 0.4600 - val_auc: 0.8300
Epoch 7/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4541 - auc: 0.8445 - val_loss: 0.4533 - val_auc: 0.8475
Epoch 8/40
1031/1031 [==========================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 61s 105ms/step - loss: 0.5417 - auc: 0.8321 - val_loss: 0.4853 - val_auc: 0.8275
Epoch 2/40
516/516 [==============================] - 53s 103ms/step - loss: 0.4575 - auc: 0.8449 - val_loss: 0.4536 - val_auc: 0.8504
Epoch 3/40
516/516 [==============================] - 53s 103ms/step - loss: 0.4544 - auc: 0.8472 - val_loss: 0.4603 - val_auc: 0.8443
Epoch 4/40
516/516 [==============================] - 53s 103ms/step - loss: 0.4529 - auc: 0.8508 - val_loss: 0.4507 - val_auc: 0.8503
Epoch 5/40
516/516 [==============================] - 54s 105ms/step - loss: 0.4523 - auc: 0.8522 - val_loss: 0.4541 - val_auc: 0.8545
Epoch 6/40
516/516 [==============================] - 53s 103ms/step - loss: 0.4519 - auc: 0.8508 - val_loss: 0.4685 - val_auc: 0.8466
Epoch 7/40
516/516 [==============================] - 53s 103ms/step - loss: 0.4529 - auc: 0.8502 - val_loss: 0.4628 - val_auc: 0.8418
Epoch 8/40
516/516 [==============================] - 5

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_13_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 53s 715ms/step - loss: 0.7935 - auc: 0.8251 - val_loss: 0.5973 - val_auc: 0.8263
Epoch 2/40
65/65 [==============================] - 46s 711ms/step - loss: 0.4577 - auc: 0.8477 - val_loss: 0.6786 - val_auc: 0.4809
Epoch 3/40
65/65 [==============================] - 45s 692ms/step - loss: 0.4486 - auc: 0.8566 - val_loss: 0.5796 - val_auc: 0.8412
Epoch 4/40
65/65 [==============================] - 45s 697ms/step - loss: 0.4425 - auc: 0.8642 - val_loss: 0.6030 - val_auc: 0.8270
Epoch 5/40
65/65 [==============================] - 45s 690ms/step - loss: 0.4402 - auc: 0.8665 - val_loss: 0.6098 - val_auc: 0.7987
Epoch 6/40
65/65 [==============================] - 46s 706ms/step - loss: 0.4316 - auc: 0.8752 - val_loss: 0.6275 - val_auc: 0.7966
Epoch 7/40
65/65 [==============================] - 46s 702ms/step - loss: 0.4291 - auc: 0.8773 - val_loss: 0.6070 - val_auc: 0.7957
Epoch 8/40
33/33 [==============================] - 1s 28ms/step - lo

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(8244, 234, 246)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([8244, 2])

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
LEARNING_RATE = 0.001
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5


opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = f'./coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 73s 63ms/step - loss: 0.4974 - auc: 0.8349 - val_loss: 0.4623 - val_auc: 0.8277
Epoch 2/40
1031/1031 [==============================] - 63s 62ms/step - loss: 0.4644 - auc: 0.8425 - val_loss: 0.5033 - val_auc: 0.8319
Epoch 3/40
1031/1031 [==============================] - 63s 61ms/step - loss: 0.4646 - auc: 0.8375 - val_loss: 0.4905 - val_auc: 0.8289
Epoch 4/40
1031/1031 [==============================] - 63s 61ms/step - loss: 0.4598 - auc: 0.8386 - val_loss: 0.4740 - val_auc: 0.8434
Epoch 5/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4561 - auc: 0.8445 - val_loss: 0.4589 - val_auc: 0.8364
Epoch 6/40
1031/1031 [==============================] - 64s 62ms/step - loss: 0.4574 - auc: 0.8416 - val_loss: 0.4534 - val_auc: 0.8509
Epoch 7/40
1031/1031 [==============================] - 62s 60ms/step - loss: 0.4537 - auc: 0.8465 - val_loss: 0.4565 - val_auc: 0.8340
Epoch 8/40
1031/1031 [==========================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 61s 106ms/step - loss: 0.5537 - auc: 0.8360 - val_loss: 0.4604 - val_auc: 0.8413
Epoch 2/40
516/516 [==============================] - 54s 104ms/step - loss: 0.4562 - auc: 0.8469 - val_loss: 0.4598 - val_auc: 0.8445
Epoch 3/40
516/516 [==============================] - 54s 105ms/step - loss: 0.4554 - auc: 0.8455 - val_loss: 0.4545 - val_auc: 0.8454
Epoch 4/40
516/516 [==============================] - 55s 107ms/step - loss: 0.4533 - auc: 0.8487 - val_loss: 0.4494 - val_auc: 0.8571
Epoch 5/40
516/516 [==============================] - 54s 104ms/step - loss: 0.4507 - auc: 0.8535 - val_loss: 0.4578 - val_auc: 0.8505
Epoch 6/40
516/516 [==============================] - 54s 104ms/step - loss: 0.4499 - auc: 0.8549 - val_loss: 0.4545 - val_auc: 0.8496
Epoch 7/40
516/516 [==============================] - 53s 104ms/step - loss: 0.4496 - auc: 0.8554 - val_loss: 0.4530 - val_auc: 0.8496
Epoch 8/40
516/516 [==============================] - 5

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_13_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 54s 725ms/step - loss: 1.1286 - auc: 0.8293 - val_loss: 0.5886 - val_auc: 0.8226
Epoch 2/40
65/65 [==============================] - 46s 701ms/step - loss: 0.4554 - auc: 0.8482 - val_loss: 0.5140 - val_auc: 0.8354
Epoch 3/40
65/65 [==============================] - 45s 699ms/step - loss: 0.4527 - auc: 0.8507 - val_loss: 0.4803 - val_auc: 0.8367
Epoch 4/40
65/65 [==============================] - 46s 704ms/step - loss: 0.4478 - auc: 0.8579 - val_loss: 0.4723 - val_auc: 0.8408
Epoch 5/40
65/65 [==============================] - 46s 705ms/step - loss: 0.4439 - auc: 0.8621 - val_loss: 0.5152 - val_auc: 0.8410
Epoch 6/40
65/65 [==============================] - 46s 701ms/step - loss: 0.4436 - auc: 0.8651 - val_loss: 0.5028 - val_auc: 0.8275
Epoch 7/40
65/65 [==============================] - 46s 709ms/step - loss: 0.4301 - auc: 0.8771 - val_loss: 0.5787 - val_auc: 0.8202
Epoch 8/40
65/65 [==============================] - 46s 703ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(8244, 234, 246)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(13700, 234, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(13700, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([13700, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = f'./coswara_mfcc_{n_mfcc}_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1713/1713 [==============================] - 113s 62ms/step - loss: 0.6524 - auc: 0.6859 - val_loss: 0.5690 - val_auc: 0.7850
Epoch 2/40
1713/1713 [==============================] - 104s 61ms/step - loss: 0.5554 - auc: 0.7893 - val_loss: 0.4781 - val_auc: 0.8463
Epoch 3/40
1713/1713 [==============================] - 104s 61ms/step - loss: 0.5301 - auc: 0.8116 - val_loss: 0.4645 - val_auc: 0.8479
Epoch 4/40
1713/1713 [==============================] - 104s 61ms/step - loss: 0.5111 - auc: 0.8259 - val_loss: 0.9079 - val_auc: 0.6908
Epoch 5/40
1713/1713 [==============================] - 104s 61ms/step - loss: 0.4933 - auc: 0.8391 - val_loss: 0.4640 - val_auc: 0.8450
Epoch 6/40
1713/1713 [==============================] - 104s 61ms/step - loss: 0.4759 - auc: 0.8522 - val_loss: 0.8244 - val_auc: 0.5806
Epoch 7/40
1713/1713 [==============================] - 104s 61ms/step - loss: 0.4515 - auc: 0.8673 - val_loss: 0.5176 - val_auc: 0.8214
Epoch 8/40
33/33 [=======================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


Epoch 1/40
857/857 [==============================] - 96s 105ms/step - loss: 0.7058 - auc: 0.6393 - val_loss: 0.5609 - val_auc: 0.8295
Epoch 2/40
857/857 [==============================] - 88s 103ms/step - loss: 0.5646 - auc: 0.7808 - val_loss: 0.5630 - val_auc: 0.7925
Epoch 3/40
857/857 [==============================] - 88s 103ms/step - loss: 0.5234 - auc: 0.8167 - val_loss: 0.5409 - val_auc: 0.8073
Epoch 4/40
857/857 [==============================] - 88s 103ms/step - loss: 0.4956 - auc: 0.8367 - val_loss: 1.0245 - val_auc: 0.6074
Epoch 5/40
857/857 [==============================] - 88s 102ms/step - loss: 0.4763 - auc: 0.8513 - val_loss: 0.6087 - val_auc: 0.7500
Epoch 6/40
33/33 [==============================] - 1s 28ms/step - loss: 0.8838 - auc: 0.5718


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_13_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


Epoch 1/40
108/108 [==============================] - 84s 716ms/step - loss: 0.9155 - auc: 0.6549 - val_loss: 0.5788 - val_auc: 0.8330
Epoch 2/40
108/108 [==============================] - 75s 698ms/step - loss: 0.5876 - auc: 0.7552 - val_loss: 0.4629 - val_auc: 0.8284
Epoch 3/40
108/108 [==============================] - 75s 691ms/step - loss: 0.5937 - auc: 0.7404 - val_loss: 0.5406 - val_auc: 0.8401
Epoch 4/40
108/108 [==============================] - 75s 696ms/step - loss: 0.5381 - auc: 0.8016 - val_loss: 0.5720 - val_auc: 0.8018
Epoch 5/40
108/108 [==============================] - 75s 694ms/step - loss: 0.4843 - auc: 0.8437 - val_loss: 2.2378 - val_auc: 0.1634
Epoch 6/40
108/108 [==============================] - 75s 690ms/step - loss: 0.4497 - auc: 0.8670 - val_loss: 1.7012 - val_auc: 0.1656
Epoch 7/40
108/108 [==============================] - 75s 696ms/step - loss: 0.4135 - auc: 0.8893 - val_loss: 0.6715 - val_auc: 0.7083
Epoch 8/40
33/33 [==============================] - 1s 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 4s 31ms/step - loss: 0.4876 - auc: 0.8483


[0.48755598068237305, 0.8483141660690308]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 28ms/step - loss: 0.4565 - auc: 0.8517


[0.4564794898033142, 0.8516815304756165]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 28ms/step - loss: 0.5258 - auc: 0.8477


[0.5258220434188843, 0.8477056622505188]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4500 - auc: 0.8530


[0.44998180866241455, 0.8529794812202454]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4545 - auc: 0.8421


[0.4545280933380127, 0.8421397805213928]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.6024 - auc: 0.8161


[0.6023785471916199, 0.8161354064941406]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4623 - auc: 0.8447


[0.4622994661331177, 0.8447111248970032]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4553 - auc: 0.8499


[0.4553108513355255, 0.849886953830719]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4495 - auc: 0.8582


[0.4494773745536804, 0.8581685423851013]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.4560 - auc: 0.8438


[0.4559778869152069, 0.8437721729278564]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.5248 - auc: 0.8222


[0.5247631072998047, 0.8222401142120361]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 1.0392 - auc: 0.8262


[1.0392462015151978, 0.8262332677841187]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.7432 - auc: 0.6704


[0.7431639432907104, 0.6704457998275757]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.8838 - auc: 0.5718


[0.8838410377502441, 0.5718249082565308]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 29ms/step - loss: 0.9275 - auc: 0.6049


[0.9274564385414124, 0.6049113869667053]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [3]:
n_mfcc = 13

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

coughvid_feature = [
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy'),
                    os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_9981.npy'),
]


### Feature preparation

#### Metadata

In [4]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,quality_1,...,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,label,sr_cough,chunk_path,num_chunk
0,00039425-7f3a-42aa-ac13-834aaa2b6b92-0,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-0.wav,3
1,00039425-7f3a-42aa-ac13-834aaa2b6b92-1,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-1.wav,3
2,00039425-7f3a-42aa-ac13-834aaa2b6b92-2,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-2.wav,3
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0.wav,3
4,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1.wav,3


In [5]:
logger.info(meta_df.shape)

2022-05-01 18:17:17,841 - COUGHVID - HYBRID MODEL CHUNKS - INFO - (84981, 43)


In [6]:
DROP_COLS = [
             'uuid',
             'num_chunk',
             'datetime',
             'chunk_path',
             'sr_cough',
             'cough_detected',
             ]

meta_df.isna().sum()

uuid                         0
datetime                     0
cough_detected               0
latitude                 33197
longitude                33197
age                      35707
gender                   31986
respiratory_condition    31986
fever_muscle_pain        31986
quality_1                80743
cough_type_1             80743
dyspnea_1                80743
wheezing_1               80743
stridor_1                80743
choking_1                80743
congestion_1             80743
nothing_1                80743
diagnosis_1              80743
severity_1               80743
quality_2                80643
cough_type_2             80651
dyspnea_2                80643
wheezing_2               80643
stridor_2                80643
choking_2                80643
congestion_2             80643
nothing_2                80643
diagnosis_2              80643
severity_2               80643
quality_3                80594
dyspnea_3                80594
wheezing_3               80594
stridor_

In [7]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3                 object
dyspnea_3 

In [8]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
cough_type_3             0
d

In [9]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [10]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-05-01 18:17:18,453 - COUGHVID - HYBRID MODEL CHUNKS - INFO - {0.0: 39866, 1.0: 13129}


In [11]:
meta_df.label.value_counts().sum()

52995

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [13]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [14]:
mfcc_feature_1 = np.load(coughvid_feature[0])
mfcc_feature_2 = np.load(coughvid_feature[1])
mfcc_feature_3 = np.load(coughvid_feature[2])
mfcc_feature_4 = np.load(coughvid_feature[3])
mfcc_feature_5 = np.load(coughvid_feature[4])

##### 2d

In [15]:
null_label = meta_df[meta_df['label'].isna() == False].index.tolist() # index of records not n/a labels
meta_df.dropna(inplace=True)
meta_df.reset_index(drop=True, inplace=True)

In [16]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [17]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-05-01 18:18:35,286 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 13129, 39866


In [18]:
# remove features of null labels
mfcc_feature_1 = mfcc_feature_1[[i for i in null_label if i < 20000]]
mfcc_feature_2 = mfcc_feature_2[[i - 20000 for i in null_label if i >= 20000 and i < 40000]]
mfcc_feature_3 = mfcc_feature_3[[i - 40000 for i in null_label if i >= 40000 and i < 60000]]
mfcc_feature_4 = mfcc_feature_4[[i - 60000 for i in null_label if i >= 60000 and i < 75000]]
mfcc_feature_5 = mfcc_feature_5[[i - 75000 for i in null_label if i >= 75000]]
mfcc_features = np.concatenate((mfcc_feature_1, mfcc_feature_2, mfcc_feature_3, mfcc_feature_4, mfcc_feature_5), axis=0)

In [19]:
# release RAM
del mfcc_feature_1
del mfcc_feature_2
del mfcc_feature_3
del mfcc_feature_4
del mfcc_feature_5

In [20]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_0)}, {len(index_1)}')

2022-05-01 18:18:52,959 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 39866, 13129


In [21]:
mfcc_features.shape

(52995, 234, 236)

In [22]:
# get features for each class
feature_0 = mfcc_features[index_0]
feature_1 = mfcc_features[index_1]
del mfcc_features

In [23]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:31893], feature_1[:10517]), axis=0)
  val = np.concatenate((feature_0[31893:35879], feature_1[10517:11824]), axis=0)
  test = np.concatenate((feature_0[35879:], feature_1[11824:]), axis=0)

  y_train = pd.concat([label_0[:31893], label_1[:10517]], ignore_index=True)
  y_val = pd.concat([label_0[31893:35879], label_1[10517:11824]], ignore_index=True)
  y_test = pd.concat([label_0[35879:], label_1[11824:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [24]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(42410, 234, 236)


In [25]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [26]:
del feature_0
del feature_1

##### 1d

In [27]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [28]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [29]:
X_train_2d = X_train_2d[20000:]
y_train_2d = y_train_2d[20000:]
X_train_2d.shape, y_train_2d.shape

((22410, 234, 236), (22410,))

In [30]:
X_train_1d = X_train_1d[20000:]
y_train_1d = y_train_1d[20000:]
X_train_1d.shape, y_train_1d.shape

((22410, 36), (22410,))

In [31]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [35]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(32410, 234, 236, 1)

In [36]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(32410, 36, 1)

In [37]:
X_train_1d.shape

(32410, 36, 1)

In [38]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [39]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [40]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [41]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [42]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [43]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
4052/4052 [==============================] - 291s 66ms/step - loss: 0.6482 - auc: 0.6747 - val_loss: 0.5728 - val_auc: 0.7674
Epoch 2/40
4052/4052 [==============================] - 259s 64ms/step - loss: 0.6330 - auc: 0.6787 - val_loss: 0.5641 - val_auc: 0.7672
Epoch 3/40
4052/4052 [==============================] - 260s 64ms/step - loss: 0.6330 - auc: 0.6742 - val_loss: 0.5689 - val_auc: 0.7399
Epoch 4/40
4052/4052 [==============================] - 261s 64ms/step - loss: 0.6326 - auc: 0.6734 - val_loss: 0.5762 - val_auc: 0.7419
Epoch 5/40
4052/4052 [==============================] - 258s 64ms/step - loss: 0.6325 - auc: 0.6742 - val_loss: 0.5743 - val_auc: 0.7650
Epoch 6/40
166/166 [==============================] - 5s 30ms/step - loss: 0.5727 - auc: 0.7633


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


Epoch 1/40
2026/2026 [==============================] - 221s 105ms/step - loss: 0.6609 - auc: 0.6920 - val_loss: 0.5600 - val_auc: 0.7791
Epoch 2/40
2026/2026 [==============================] - 205s 101ms/step - loss: 0.6306 - auc: 0.6914 - val_loss: 0.5626 - val_auc: 0.7626
Epoch 3/40
2026/2026 [==============================] - 207s 102ms/step - loss: 0.6299 - auc: 0.6904 - val_loss: 0.5740 - val_auc: 0.7660
Epoch 4/40
2026/2026 [==============================] - 208s 103ms/step - loss: 0.6286 - auc: 0.6934 - val_loss: 0.5763 - val_auc: 0.7655
Epoch 5/40
2026/2026 [==============================] - 210s 104ms/step - loss: 0.6273 - auc: 0.6966 - val_loss: 0.5764 - val_auc: 0.7795
Epoch 6/40
166/166 [==============================] - 5s 29ms/step - loss: 0.5760 - auc: 0.7705


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_13_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


Epoch 1/40
254/254 [==============================] - 182s 681ms/step - loss: 0.7860 - auc: 0.6775 - val_loss: 0.6446 - val_auc: 0.7393
Epoch 2/40
254/254 [==============================] - 171s 674ms/step - loss: 0.6299 - auc: 0.6902 - val_loss: 0.6365 - val_auc: 0.7513
Epoch 3/40
254/254 [==============================] - 171s 675ms/step - loss: 0.6269 - auc: 0.6985 - val_loss: 0.5817 - val_auc: 0.7678
Epoch 4/40
254/254 [==============================] - 171s 672ms/step - loss: 0.6231 - auc: 0.7068 - val_loss: 0.5741 - val_auc: 0.7739
Epoch 5/40
254/254 [==============================] - 170s 669ms/step - loss: 0.6201 - auc: 0.7128 - val_loss: 0.5728 - val_auc: 0.7785
Epoch 6/40
254/254 [==============================] - 170s 669ms/step - loss: 0.6178 - auc: 0.7168 - val_loss: 0.5896 - val_auc: 0.7738
Epoch 7/40
254/254 [==============================] - 170s 669ms/step - loss: 0.6147 - auc: 0.7209 - val_loss: 0.5774 - val_auc: 0.7778
Epoch 8/40
254/254 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [32]:
X_train_2d.shape

(22410, 234, 236)

In [33]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(22410, 234, 236, 1)

In [34]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(22410, 36, 1)

In [35]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [36]:
y_train_2d.shape

TensorShape([22410, 2])

In [37]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [38]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [41]:
fp = f'./coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [42]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
2802/2802 [==============================] - 202s 66ms/step - loss: 0.7172 - auc: 0.5166 - val_loss: 0.6488 - val_auc: 0.7423
Epoch 2/40
2802/2802 [==============================] - 182s 65ms/step - loss: 0.6952 - auc: 0.5289 - val_loss: 0.6450 - val_auc: 0.7452
Epoch 3/40
2802/2802 [==============================] - 181s 65ms/step - loss: 0.6941 - auc: 0.5290 - val_loss: 0.6872 - val_auc: 0.5799
Epoch 4/40
2802/2802 [==============================] - 184s 66ms/step - loss: 0.6944 - auc: 0.5225 - val_loss: 0.6441 - val_auc: 0.7687
Epoch 5/40
2802/2802 [==============================] - 181s 65ms/step - loss: 0.6946 - auc: 0.5239 - val_loss: 0.6531 - val_auc: 0.7433
Epoch 6/40
2802/2802 [==============================] - 182s 65ms/step - loss: 0.6942 - auc: 0.5240 - val_loss: 0.7020 - val_auc: 0.5541
Epoch 7/40
2802/2802 [==============================] - 182s 65ms/step - loss: 0.6937 - auc: 0.5240 - val_loss: 0.6696 - val_auc: 0.6663
Epoch 8/40
2802/2802 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


Epoch 1/40
1401/1401 [==============================] - 158s 107ms/step - loss: 0.7265 - auc: 0.5016 - val_loss: 0.6770 - val_auc: 0.6558
Epoch 2/40
1401/1401 [==============================] - 148s 106ms/step - loss: 0.6939 - auc: 0.5247 - val_loss: 0.6678 - val_auc: 0.7581
Epoch 3/40
1401/1401 [==============================] - 147s 105ms/step - loss: 0.6926 - auc: 0.5348 - val_loss: 0.6158 - val_auc: 0.7533
Epoch 4/40
1401/1401 [==============================] - 148s 105ms/step - loss: 0.6936 - auc: 0.5280 - val_loss: 0.6770 - val_auc: 0.5673
Epoch 5/40
1401/1401 [==============================] - 148s 106ms/step - loss: 0.6934 - auc: 0.5266 - val_loss: 0.6784 - val_auc: 0.6381
Epoch 6/40
1401/1401 [==============================] - 148s 106ms/step - loss: 0.6933 - auc: 0.5275 - val_loss: 0.6741 - val_auc: 0.7105
Epoch 7/40
1401/1401 [==============================] - 149s 106ms/step - loss: 0.6899 - auc: 0.5501 - val_loss: 0.6337 - val_auc: 0.7715
Epoch 8/40
1401/1401 [============

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_13_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


Epoch 1/40
176/176 [==============================] - 130s 691ms/step - loss: 0.8122 - auc: 0.4782 - val_loss: 0.6717 - val_auc: 0.7434
Epoch 2/40
176/176 [==============================] - 120s 683ms/step - loss: 0.6938 - auc: 0.5261 - val_loss: 0.6684 - val_auc: 0.7401
Epoch 3/40
176/176 [==============================] - 121s 686ms/step - loss: 0.6957 - auc: 0.5264 - val_loss: 0.6682 - val_auc: 0.7577
Epoch 4/40
176/176 [==============================] - 120s 681ms/step - loss: 0.6929 - auc: 0.5242 - val_loss: 0.6674 - val_auc: 0.7602
Epoch 5/40
176/176 [==============================] - 120s 681ms/step - loss: 0.6927 - auc: 0.5304 - val_loss: 0.6759 - val_auc: 0.7075
Epoch 6/40
176/176 [==============================] - 120s 681ms/step - loss: 0.6922 - auc: 0.5288 - val_loss: 0.6676 - val_auc: 0.7380
Epoch 7/40
176/176 [==============================] - 119s 679ms/step - loss: 0.6919 - auc: 0.5334 - val_loss: 0.6710 - val_auc: 0.7419
Epoch 8/40
176/176 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


## SMOTE

In [32]:
X_train_2d.shape

(22410, 234, 236)

In [33]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(23786, 234, 236, 1)

In [34]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(23786, 36, 1)

In [35]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [36]:
y_train_2d.shape

TensorShape([23786, 2])

In [37]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [38]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()

In [40]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [41]:
fp = f'./coughvid_hybrid_mfcc_{n_mfcc}_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [42]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
2974/2974 [==============================] - 203s 62ms/step - loss: 0.7199 - auc: 0.4957 - val_loss: 0.6667 - val_auc: 0.7573
Epoch 2/40
2974/2974 [==============================] - 184s 62ms/step - loss: 0.6964 - auc: 0.5040 - val_loss: 0.6878 - val_auc: 0.5621
Epoch 3/40
2974/2974 [==============================] - 186s 62ms/step - loss: 0.6969 - auc: 0.4971 - val_loss: 0.6690 - val_auc: 0.6749
Epoch 4/40
2974/2974 [==============================] - 186s 62ms/step - loss: 0.6963 - auc: 0.5008 - val_loss: 0.6915 - val_auc: 0.5061
Epoch 5/40
2974/2974 [==============================] - 186s 62ms/step - loss: 0.6961 - auc: 0.4997 - val_loss: 0.7225 - val_auc: 0.3206
Epoch 6/40
166/166 [==============================] - 5s 30ms/step - loss: 0.6791 - auc: 0.7458


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_13_hybrid_model_40_8_0.001/assets


Epoch 1/40
1487/1487 [==============================] - 166s 106ms/step - loss: 0.7454 - auc: 0.5297 - val_loss: 0.7069 - val_auc: 0.4746
Epoch 2/40
1487/1487 [==============================] - 154s 104ms/step - loss: 0.6958 - auc: 0.5033 - val_loss: 0.6994 - val_auc: 0.4301
Epoch 3/40
1487/1487 [==============================] - 154s 104ms/step - loss: 0.6952 - auc: 0.5081 - val_loss: 0.6966 - val_auc: 0.5098
Epoch 4/40
1487/1487 [==============================] - 154s 104ms/step - loss: 0.6955 - auc: 0.5054 - val_loss: 0.6717 - val_auc: 0.7391
Epoch 5/40
1487/1487 [==============================] - 154s 104ms/step - loss: 0.6951 - auc: 0.5080 - val_loss: 0.6648 - val_auc: 0.7506
Epoch 6/40
1487/1487 [==============================] - 154s 104ms/step - loss: 0.6951 - auc: 0.5098 - val_loss: 0.6952 - val_auc: 0.4842
Epoch 7/40
1487/1487 [==============================] - 154s 103ms/step - loss: 0.6953 - auc: 0.5008 - val_loss: 0.7012 - val_auc: 0.3448
Epoch 8/40
1487/1487 [============

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_13_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_13_hybrid_model_40_64_0.001/assets


Epoch 1/40
186/186 [==============================] - 138s 694ms/step - loss: 0.8490 - auc: 0.4775 - val_loss: 0.6975 - val_auc: 0.2606
Epoch 2/40
186/186 [==============================] - 127s 682ms/step - loss: 0.6940 - auc: 0.5097 - val_loss: 0.7218 - val_auc: 0.2601
Epoch 3/40
186/186 [==============================] - 127s 681ms/step - loss: 0.6912 - auc: 0.5386 - val_loss: 0.7277 - val_auc: 0.2648
Epoch 4/40
186/186 [==============================] - 127s 682ms/step - loss: 0.6889 - auc: 0.5548 - val_loss: 0.6925 - val_auc: 0.5216
Epoch 5/40
186/186 [==============================] - 127s 681ms/step - loss: 0.6857 - auc: 0.5704 - val_loss: 0.7378 - val_auc: 0.3700
Epoch 6/40
186/186 [==============================] - 127s 686ms/step - loss: 0.6834 - auc: 0.5779 - val_loss: 0.6808 - val_auc: 0.5668
Epoch 7/40
186/186 [==============================] - 126s 678ms/step - loss: 0.6805 - auc: 0.5879 - val_loss: 0.6568 - val_auc: 0.6367
Epoch 8/40
186/186 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coughvid_mfcc_13_hybrid_model_40_128_0.001/assets


## Evaluation

In [32]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [33]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [34]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [35]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 14s 29ms/step - loss: 0.6781 - auc: 0.6789


[0.6781483292579651, 0.6789036393165588]

In [36]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.7470 - auc: 0.3483


[0.7469555139541626, 0.3483121693134308]

In [37]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.6481 - auc: 0.7246


[0.6480864882469177, 0.7245969772338867]

In [38]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 30ms/step - loss: 0.6510 - auc: 0.6568


[0.6509571075439453, 0.6567890644073486]

In [39]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 30ms/step - loss: 0.7145 - auc: 0.3956


[0.7144867181777954, 0.39555874466896057]

### Raw features

In [40]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [41]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.5902 - auc: 0.7611


[0.5902118682861328, 0.7611127495765686]

In [42]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.5816 - auc: 0.7665


[0.5815765857696533, 0.7665261626243591]

In [43]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.5771 - auc: 0.7754


[0.5770948529243469, 0.7754217386245728]

In [44]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.5760 - auc: 0.7705


[0.5760442614555359, 0.7705352306365967]

In [45]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.5727 - auc: 0.7633


[0.5726699233055115, 0.7633148431777954]

### SMOTE

In [46]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.9896 - auc: 0.4170


[0.9896187782287598, 0.4169788360595703]

In [47]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.7423 - auc: 0.3994


[0.7423133254051208, 0.3994048833847046]

In [48]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.6779 - auc: 0.5604


[0.6779326796531677, 0.5603798031806946]

In [49]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.6874 - auc: 0.6068


[0.6874495148658752, 0.6068428754806519]

In [50]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coughvid_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 29ms/step - loss: 0.6791 - auc: 0.7458


[0.679050087928772, 0.7457658052444458]